# directory

In [1]:
import os

def files_in_directory(directory_path):
    # List all items in the directory
    all_items = os.listdir(directory_path)
    # Filter out only files (not directories)
    files = [item for item in all_items if os.path.isfile(os.path.join(directory_path, item))]
    return files

def dir_in_directory(directory_path):
    # List all items in the directory
    all_items = os.listdir(directory_path)
    # Filter out only files (not directories)
    files = [item for item in all_items if os.path.isdir(os.path.join(directory_path, item))]
    return files

In [ ]:
directory_path = 'signal/train/train/'
print(files_in_directory(directory_path))
print(len(files_in_directory(directory_path)))
print()

directory_path = 'signal/test/test_segment/'
print(dir_in_directory(directory_path))
print(len(dir_in_directory(directory_path)))
print()

directory_path = 'signal/test/test_segment/test010/'
print(files_in_directory(directory_path))
print(len(files_in_directory(directory_path)))

In [2]:
def train_path_files(directory_path):
    # List all items in the directory
    all_items = os.listdir(directory_path)
    # Filter out only files (not directories)
    files = [os.path.join(directory_path, item) for item in all_items if os.path.isfile(os.path.join(directory_path, item))]
    return files

def test_path_files(directory_path):
    # List all items in the directory
    all_items = os.listdir(directory_path)
    # Filter out only files (not directories)
    files = [os.path.join(directory_path, item) for item in all_items if os.path.isdir(os.path.join(directory_path, item))]
    final_files = []
    for f in files:
        final_files.extend([os.path.join(f, item) for item in files_in_directory(f)])
    return final_files

In [3]:
train_file_ls = train_path_files('signal/train/train/')
test_file_ls = test_path_files('signal/test/test_segment/')

len(train_file_ls), len(test_file_ls)

(83, 7970)

In [4]:
test_file_ls.sort()
test_file_ls[:5]

['signal/test/test_segment/test001/test001_00000.csv',
 'signal/test/test_segment/test001/test001_00001.csv',
 'signal/test/test_segment/test001/test001_00002.csv',
 'signal/test/test_segment/test001/test001_00003.csv',
 'signal/test/test_segment/test001/test001_00004.csv']

# format data

In [5]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
from tqdm import tqdm

In [6]:
lbsl_ls = []
X_ls = []
for i in tqdm(range(len(train_file_ls))):
    df_train = pd.read_csv(train_file_ls[i])
    for j in range(int(df_train.shape[0] / 1920)):
        label_sleep_j = df_train.loc[j,'Sleep_Stage']
        train_chunk = np.array(df_train.loc[j*1920:((j+1)*1920)- 1].drop(columns='Sleep_Stage'))

        X_ls.append(list(train_chunk.reshape(-1)))
        
        lbsl_ls.append(label_sleep_j)

100%|██████████| 83/83 [02:01<00:00,  1.46s/it]


In [ ]:
X_ls[:2]

In [17]:
lbsl_ls[:5]

['W', 'W', 'W', 'W', 'W']

In [7]:
Xtest_ls = []

for i in tqdm(range(len(test_file_ls))):
    df_test = pd.read_csv(test_file_ls[i])

    test_chunk = np.array(df_test)

    Xtest_ls.append(list(test_chunk.reshape(-1)))
    

100%|██████████| 7970/7970 [00:57<00:00, 137.42it/s]


In [ ]:
Xtest_ls[:2]

# prepare data

In [9]:
pd.Series(lbsl_ls).value_counts()

W    65072
N     1535
Name: count, dtype: int64

In [30]:
import random

n_ls = [i for i in range(len(lbsl_ls)) if lbsl_ls[i] == 'N']
w_ls = [i for i in range(len(lbsl_ls)) if lbsl_ls[i] == 'W']

X_N = [X_ls[i] for i in n_ls]
y_N = [lbsl_ls[i] for i in n_ls]

random.seed(0)
samp_w_ind = random.sample(w_ls, len(n_ls))
samp_w_ind

X_W = [X_ls[i] for i in samp_w_ind]
y_W = [lbsl_ls[i] for i in samp_w_ind]

In [36]:
X = []
X.extend(X_W)
X.extend(X_N)

y = []
y.extend(y_W)
y.extend(y_N)

len(X), len(y)

(3070, 3070)

In [37]:
pd.Series(y).value_counts()

W    1535
N    1535
Name: count, dtype: int64

In [40]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score

In [41]:
# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.25, random_state=888, stratify=y)

In [50]:
X_train, X_val, y_train, y_val = train_test_split(X_ls, lbsl_ls, test_size=0.25, random_state=888, stratify=lbsl_ls)

# train

In [51]:
from catboost import CatBoostClassifier

clf = CatBoostClassifier()


clf.fit(X_train, y_train,
        # cat_features=cat_features,
)

print('CatBoost model is fitted: ' + str(clf.is_fitted()))

Learning rate set to 0.05473
0:	learn: 0.5708984	total: 407ms	remaining: 6m 46s
1:	learn: 0.4916878	total: 606ms	remaining: 5m 2s
2:	learn: 0.4245593	total: 801ms	remaining: 4m 26s
3:	learn: 0.3616712	total: 994ms	remaining: 4m 7s
4:	learn: 0.3109988	total: 1.19s	remaining: 3m 56s
5:	learn: 0.2602039	total: 1.38s	remaining: 3m 48s
6:	learn: 0.2259746	total: 1.58s	remaining: 3m 43s
7:	learn: 0.1977160	total: 1.77s	remaining: 3m 39s
8:	learn: 0.1667658	total: 1.97s	remaining: 3m 36s
9:	learn: 0.1484888	total: 2.16s	remaining: 3m 34s
10:	learn: 0.1310599	total: 2.36s	remaining: 3m 31s
11:	learn: 0.1194461	total: 2.55s	remaining: 3m 30s
12:	learn: 0.1087138	total: 2.75s	remaining: 3m 28s
13:	learn: 0.0959868	total: 2.94s	remaining: 3m 27s
14:	learn: 0.0868542	total: 3.14s	remaining: 3m 25s
15:	learn: 0.0794933	total: 3.33s	remaining: 3m 24s
16:	learn: 0.0739438	total: 3.53s	remaining: 3m 23s
17:	learn: 0.0692676	total: 3.72s	remaining: 3m 23s
18:	learn: 0.0642437	total: 3.92s	remaining: 3m

In [52]:
y_val_pred = model.predict(X_val)

accuracy = accuracy_score(y_val, y_val_pred)
print(f'Validation Accuracy: {accuracy}')
print(classification_report(y_val, y_val_pred))

Validation Accuracy: 0.9731563776122988
              precision    recall  f1-score   support

           N       0.46      0.97      0.62       384
           W       1.00      0.97      0.99     16268

    accuracy                           0.97     16652
   macro avg       0.73      0.97      0.81     16652
weighted avg       0.99      0.97      0.98     16652



In [42]:
from catboost import CatBoostClassifier, Pool

best_params = {'iterations': 1981,
 'border_count': 179,
 'bagging_temperature': 0.5321224591936926,
 'depth': 15,
 'learning_rate': 0.057594837270947837,
 'l2_leaf_reg': 0.03770872402240005,
 'max_leaves': 45}

model = CatBoostClassifier(
        **best_params,
        random_seed=888,
        task_type='GPU',
        use_best_model=True,
        eval_metric='Accuracy',
        grow_policy='Lossguide',
        boosting_type='Plain'
    )
train_pool = Pool(X_train, y_train)
val_pool = Pool(X_val, y_val)

model.fit(train_pool, eval_set=val_pool, verbose=False)
y_val_pred = model.predict(X_val)

accuracy = accuracy_score(y_val, y_val_pred)
print(f'Validation Accuracy: {accuracy}')
print(classification_report(y_val, y_val_pred))

Validation Accuracy: 0.9765625
              precision    recall  f1-score   support

           N       0.98      0.97      0.98       384
           W       0.97      0.98      0.98       384

    accuracy                           0.98       768
   macro avg       0.98      0.98      0.98       768
weighted avg       0.98      0.98      0.98       768



# predict

In [53]:
y_test_pred = model.predict(Xtest_ls)
y_test_pred

array(['W', 'W', 'W', ..., 'W', 'W', 'W'], dtype=object)

In [54]:
sp_sb = pd.read_csv('signal/sample_submission.csv')
sp_sb

,id,labels
0,test001_00000,W
1,test001_00001,W
2,test001_00002,W
3,test001_00003,NaN
4,test001_00004,NaN
...,...,...
7965,test010_00857,NaN
7966,test010_00858,NaN
7967,test010_00859,NaN
7968,test010_00860,NaN


In [55]:
sp_sb.loc[3:, 'labels'] = y_test_pred[3:]
sp_sb

,id,labels
0,test001_00000,W
1,test001_00001,W
2,test001_00002,W
3,test001_00003,W
4,test001_00004,W
...,...,...
7965,test010_00857,W
7966,test010_00858,W
7967,test010_00859,W
7968,test010_00860,W


In [56]:
sp_sb['labels'].value_counts()

labels
W    7838
N     132
Name: count, dtype: int64

In [57]:
# sp_sb.to_csv('signal_catbost_downSamp.csv', index=False)
sp_sb.to_csv('signal_catbost_notTuneParam.csv', index=False)